In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent.parent))

In [2]:
import time
import triton
import triton.language as tl
import torch
import torch.nn.functional as F
import pandas as pd
from conv_gemm.triton_kernels.int8.col2img_int8_kernel import col2img_int32_kernel

In [3]:
def col2img_int32(
    cols_i32: torch.Tensor,
    N: int, Cin: int,
    H: int, W: int,
    Kh: int, Kw: int,
    Sh: int, Sw: int,
    Ph: int, Pw: int,
    Dh: int, Dw: int,
    BLOCK_M: int,
    BLOCK_K: int,
    num_warps: int = 4,
    num_stages: int = 2,
):

    assert cols_i32.is_cuda
    assert cols_i32.dtype == torch.int32
    cols_i32 = cols_i32.contiguous()

    Ho = (H + 2 * Ph - Dh * (Kh - 1) - 1) // Sh + 1
    Wo = (W + 2 * Pw - Dw * (Kw - 1) - 1) // Sw + 1
    M = N * Ho * Wo
    K = Cin * Kh * Kw

    assert cols_i32.shape == (M, K), f"cols shape {cols_i32.shape}, expected {(M, K)}"

    x_i32 = torch.zeros((N, Cin, H, W), device=cols_i32.device, dtype=torch.int32)
    sN, sC, sH, sW = x_i32.stride()

    grid = (triton.cdiv(M, BLOCK_M), triton.cdiv(K, BLOCK_K))

    col2img_int32_kernel[grid](
        cols_i32, x_i32,
        N, Cin, H, W,
        Kh, Kw, Sh, Sw, Ph, Pw, Dh, Dw,
        Ho, Wo,
        sN, sC, sH, sW,
        K,
        BLOCK_M=BLOCK_M,
        BLOCK_K=BLOCK_K,
        num_warps=num_warps,
        num_stages=num_stages,
    )
    return x_i32

In [4]:
@torch.no_grad()
def bench_once_col2img_int32_vs_fold(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    BLOCK_M,
    BLOCK_K,
    num_warps,
    num_stages,
    iters=100,
    device="cuda",
):

    Ho = (H + 2 * Ph - Dh * (Kh - 1) - 1) // Sh + 1
    Wo = (W + 2 * Pw - Dw * (Kw - 1) - 1) // Sw + 1
    assert Ho > 0 and Wo > 0, f"Invalid Ho,Wo = {(Ho, Wo)}"

    M = N * Ho * Wo
    K = Cin * Kh * Kw

    cols_i32 = torch.randint(
        low=-128, high=127,
        size=(M, K),
        device=device,
        dtype=torch.int32,
    )

    # Torch  F.fold 
    cols_f = cols_i32.float().half()
    cols_fold = cols_f.view(N, Ho * Wo, K).permute(0, 2, 1).contiguous()  # [N, K, L]

    def _call_torch():
        return F.fold(
            cols_fold,
            output_size=(H, W),
            kernel_size=(Kh, Kw),
            dilation=(Dh, Dw),
            padding=(Ph, Pw),
            stride=(Sh, Sw),
        )  


    for _ in range(5):
        x_ref = _call_torch()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        x_ref = _call_torch()
    torch.cuda.synchronize()
    t_torch = (time.perf_counter() - t0) / iters

    # Triton
    def _call_triton():
        x_i32 = col2img_int32(
            cols_i32,
            N, Cin,
            H, W,
            Kh, Kw,
            Sh, Sw,
            Ph, Pw,
            Dh, Dw,
            BLOCK_M=BLOCK_M,
            BLOCK_K=BLOCK_K,
            num_warps=num_warps,
            num_stages=num_stages,
        )
        return x_i32


    for _ in range(5):
        x_i32 = _call_triton()
    torch.cuda.synchronize()

    t0 = time.perf_counter()
    for _ in range(iters):
        x_i32 = _call_triton()
    torch.cuda.synchronize()
    t_triton = (time.perf_counter() - t0) / iters

    # Error Triton INT32 vs Torch FP16
    diff = (x_i32.float() - x_ref.float()).abs()
    max_abs_err16 = diff.max().item()
    mean_abs_err16 = diff.mean().item()

    # bandwidth 
    bytes_moved = (cols_i32.numel() + x_i32.numel()) * 4.0  # int32
    bw_triton = bytes_moved / t_triton / 1e9
    bw_torch  = bytes_moved / t_torch  / 1e9

    return {
        "N": N,
        "Cin": Cin,
        "H": H,
        "W": W,
        "Kh": Kh,
        "Kw": Kw,
        "M": M,
        "K": K,
        "BLOCK_M": BLOCK_M,
        "BLOCK_N": 0,  
        "BLOCK_K": BLOCK_K,
        "num_warps": num_warps,
        "num_stages": num_stages,
        "t_triton_ms": t_triton * 1e3,
        "t_torch16_ms": t_torch * 1e3,
        "speed_vs_torch16": t_torch / t_triton,
        "bw_triton_GBs": bw_triton,
        "bw_torch16_GBs": bw_torch,
        "max_abs_err": max_abs_err16,
        "mean_abs_err": mean_abs_err16,
    }


In [5]:
@torch.no_grad()
def tune_col2img_int32_tiles_for_shape(
    N, Cin, H, W,
    Kh, Kw,
    Sh, Sw,
    Ph, Pw,
    Dh, Dw,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(1, 2, 4, 8),
    stages=(2, 3, 4),
    iters=200,
    device="cuda",
):
    records = []
    for BM in blocks_M:
        for BK in blocks_K:
            for Wp in warps:
                for S in stages:
                    try:
                        rec = bench_once_col2img_int32_vs_fold(
                            N, Cin, H, W,
                            Kh, Kw,
                            Sh, Sw,
                            Ph, Pw,
                            Dh, Dw,
                            BLOCK_M=BM,
                            BLOCK_K=BK,
                            num_warps=Wp,
                            num_stages=S,
                            iters=iters,
                            device=device,
                        )
                    except RuntimeError as e:
                        print(f"[SKIP] BM={BM}, BK={BK}, W={Wp}, S={S}: {e}")
                        continue

                    print(
                        f"BM={BM}, BK={BK}, W={Wp}, S={S}: "
                        f"t_triton={rec['t_triton_ms']:.3f} ms, "
                        f"speed_vs_torch={rec['speed_vs_torch16']:.3f}x, "
                    )
                    records.append(rec)

    if not records:
        raise RuntimeError("No valid tile configs found for this COL2IMG shape")

    df = pd.DataFrame(records)
    return df

In [6]:
df_c2i = tune_col2img_int32_tiles_for_shape(
    N=16, Cin=1, H=256, W=256,
    Kh=11, Kw=11,
    Sh=1, Sw=1,
    Ph=5, Pw=5,
    Dh=1, Dw=1,
    blocks_M=(32, 64, 128),
    blocks_K=(32, 64, 128),
    warps=(2, 4, 8),
    stages=(2, 3),
    iters=200,
    device="cuda",
)

BM=32, BK=32, W=2, S=2: t_triton=2.151 ms, speed_vs_torch=1.063x, 
BM=32, BK=32, W=2, S=3: t_triton=2.049 ms, speed_vs_torch=1.092x, 
BM=32, BK=32, W=4, S=2: t_triton=1.595 ms, speed_vs_torch=1.471x, 
BM=32, BK=32, W=4, S=3: t_triton=1.583 ms, speed_vs_torch=1.431x, 
BM=32, BK=32, W=8, S=2: t_triton=1.722 ms, speed_vs_torch=1.319x, 
BM=32, BK=32, W=8, S=3: t_triton=1.741 ms, speed_vs_torch=1.301x, 
BM=32, BK=64, W=2, S=2: t_triton=4.781 ms, speed_vs_torch=0.463x, 
BM=32, BK=64, W=2, S=3: t_triton=5.096 ms, speed_vs_torch=0.499x, 
BM=32, BK=64, W=4, S=2: t_triton=2.135 ms, speed_vs_torch=1.146x, 
BM=32, BK=64, W=4, S=3: t_triton=2.184 ms, speed_vs_torch=1.111x, 
BM=32, BK=64, W=8, S=2: t_triton=1.840 ms, speed_vs_torch=1.296x, 
BM=32, BK=64, W=8, S=3: t_triton=1.798 ms, speed_vs_torch=1.330x, 
BM=32, BK=128, W=2, S=2: t_triton=6.349 ms, speed_vs_torch=0.377x, 
BM=32, BK=128, W=2, S=3: t_triton=6.372 ms, speed_vs_torch=0.376x, 
BM=32, BK=128, W=4, S=2: t_triton=5.392 ms, speed_vs_torch=0

In [7]:
cols = [
    "BLOCK_M", "BLOCK_K",
    "num_warps", "num_stages",
    "t_triton_ms", "t_torch16_ms",
    "speed_vs_torch16",
     "mean_abs_err"
]

df_c2i_filtered = df_c2i[cols].sort_values("speed_vs_torch16", ascending=False).head(5).reset_index(drop=True)
df_c2i_filtered

,BLOCK_M,BLOCK_K,num_warps,num_stages,t_triton_ms,t_torch16_ms,speed_vs_torch16,mean_abs_err
0,32,32,4,2,1.595475,2.346608,1.470790,0.005841
1,32,32,4,3,1.582950,2.264513,1.430565,0.005771
2,64,32,8,3,1.822141,2.502242,1.373243,0.005835
3,64,32,8,2,1.836549,2.511122,1.367304,0.005736
4,32,64,8,3,1.797709,2.390389,1.329686,0.005742


In [8]:
INT8_COL2IMG_BEST_BLOCK_M = 32
INT8_COL2IMG_BEST_BLOCK_N = 0
INT8_COL2IMG_BEST_BLOCK_K = 32
INT8_COL2IMG_BEST_WARPS = 4
INT8_COL2IMG_BEST_STAGES = 2